# Game of Thrones

* Get the lists of character names
* Use API's to download all the character pages
* Create a network by using those pages

In [11]:
#Import packages
%matplotlib inline
import re
import io
import os
import urllib2
import networkx as nx
import community
import json
import glob
import pickle
import numpy as np
import operator
import matplotlib.pyplot as plt
from matplotlib import pyplot
import math

## Use API's to get lists of characters

In [12]:
#Get namelist, idlist and url list of the characters
base="http://gameofthrones.wikia.com/api/v1/Articles/List?category=Characters&"
#Insert how many characters we want. This is just all of them.
limit="limit=1057"
listquery="%s%s" %(base,limit)
textloadlist= json.loads(urllib2.urlopen(listquery).read())
namelist=[]
idlist=[]
urllist=[]
for i in range(len(textloadlist['items'])):
    namelist.append(textloadlist['items'][i]['title'])
    idlist.append(textloadlist['items'][i]['id'])
    urllist.append(textloadlist['items'][i]['url'][6:])

## Save the lists as textfiles for easy use in the future

In [4]:
f = io.open("char_names.txt", 'w+',encoding="utf-8") 
for item in namelist:
    f.write("%s\n" % item)
f.close()

In [5]:
f = io.open("url_names.txt", 'w+',encoding="utf-8") 
for item in urllist:
    f.write("%s\n" % item)
f.close()

In [6]:
f = io.open("id_list.txt", 'w+',encoding="utf-8") 
for item in idlist:
    f.write("%s\n" % unicode(item))
f.close()

In [13]:
print namelist[:10]
print urllist[:10]
print idlist[:10]

[u'Aemma Arryn', u'Aemon', u'Aemond Targaryen', u'Aggar', u'Aggo', u'Akho', u'Allo', u'Amaya', u'Anara', u'Andros']
[u'Aemma_Arryn', u'Aemon', u'Aemond_Targaryen', u'Aggar', u'Aggo', u'Akho', u'Allo', u'Amaya', u'Anara', u'Andros']
[49600, 2206, 30718, 10245, 6660, 39242, 21870, 36172, 29843, 30956]


In [7]:
print len(namelist)

1057


## Download the wiki page for every Game of Thrones chacter

In [11]:
filedir=glob.glob('char_pages/*')
filelist=[]
for name in filedir:
    filelist.append(name[11:-4].lower())
# Get the wiki page for each character
for i in range(len(namelist)):
    if namelist[i].replace("/", "_").lower() in filelist:
        continue
    #Use the id to access the different pages
    query="http://gameofthrones.wikia.com/api/v1/Articles/AsJson?id=%s" %idlist[i]
    query=query.encode('utf-8')
    response= urllib2.urlopen(query)
    source= response.read()
    textload=json.loads(source)
    
    #Write the content of the article
    filename="%s.txt" % namelist[i]
    #Do this since / can't be used in a file name
    filename=filename.replace("/", "_")
    f = io.open(filename, 'w+',encoding="utf-8") 
    f.write(unicode(textload['content']))
    f.close()
    destname="char_pages/%s" % filename
    os.rename(filename, destname) 
print "done"

done


## Create a graph/network of all the characters. Do this based on who link to each other.

In [12]:
#Create both a directed graph and an undirected graph
G=nx.DiGraph()
H=nx.Graph()
G.add_nodes_from(namelist)
H.add_nodes_from(namelist)
for i in range(len(namelist)):
    name=namelist[i].replace("/", "_")
    url=urllist[i]
    for p in range(len(namelist)):
        if p==i:
            continue
        filename="char_pages/%s.txt" % namelist[p].replace("/", "_")
        f = io.open(filename, 'r' ,encoding='utf-8')
        readfile=f.read()
        regexname=r"\/" + re.escape(url) + r"\b"
        match=re.search(regexname, readfile)   
        if match:
            G.add_edge(namelist[p], namelist[i])
            H.add_edge(namelist[p], namelist[i])
print "done"

done


In [26]:
#Save the graphs as gpickle files for easy use in the future

#nx.write_gpickle(G, "Digraph_GOT.gpickle")
#nx.write_gpickle(H, "Udigraph_GOT.gpickle")

In [14]:
#Print some simple stats to show that the graph were successfully created
print "The number of edges in the graph is %d" % G.number_of_edges()
print "The number of nodes in the graph is %d" % G.number_of_nodes()
print "The number of edges in the graph is %d" % H.number_of_edges()
print "The number of nodes in the graph is %d" % H.number_of_nodes()

The number of edges in the graph is 9165
The number of nodes in the graph is 1057
The number of edges in the graph is 6759
The number of nodes in the graph is 1057
